<a href="https://colab.research.google.com/github/root-epifit/Graphs/blob/main/Week_3/HW3/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Построение рекомендательной системы

В этом нотбуке мы выполним наш проект — построим рекомендательную систему, основанную на методах, которые мы обсудили на занятии.

Для этого мы будем использовать данные от Амазон. Граф размещен в отдельном файле, он загружается в первом блоке.

Нам нужно будет реализовать четыре метода, обсуждавшиеся на занятии, и протестировать их. Но общий подход во всех четырех методах один и тот же: 
1. мы фиксируем вершину (в коде ниже это переменная `query`); 
2. удаляем некоторые смежные с ней ребра (в коде ниже это список `samp`); 
3. вычисляем специально определенное расстояние между нашей вершиной и всеми остальными (методы различаются как раз выбором расстояния);
4. выбираем вершины с наименьшим расстоянием до выбранной, это те вершины, в которые метод предлагает провести ребра;
5. сравниваем предложенные методом ребра с удаленными, чем больше совпадений, тем лучше сработал метод.

Вспомогательные шаги уже реализованы ниже. Шаг 2 реализован в функции `generate_dict`. Шаги 4-5 реализованы в функции `check_answer`. Вам нужно реализовать только шаги 3 для всех методов.

В первом методе, который нужно реализовать, расстоянием является просто число общих соседей. Во втором методе нужно будет посчитать усеченные моменты достижения из выбранной вершины. Их мы подсчитываем приближенно, запуская случайное блуждание несколько раз. Сначала нужно реализовать функцию для одного случайного блуждания, затем функцию для приближенного вычисления усеченных моментов достижения. Длину блуждания мы фиксируем равной 10. В третьем методе нужно посчитать усеченные моменты достижения в вершину. Для них у нас есть рекуррентная формула. Наконец, в последнем методе нужно просто посчитать суммы результатов двух предыдущих методов.

Мы подробно обсуждали все эти методы на занятии. Ниже вы также можете найти поясняющие комментарии.

Добавьте ваше решение между строками "BEGIN SOLUTION" и "END SOLUTION". Желательно не менять остальной код.

---
**Правила сдачи и оценивания.** Это задание является проектом курса, оно оценивается в 30 баллов плюс 15 бонусных баллов.

Дедлайн по выполнению проекта --- **1 марта в 19:00**. Решения нужно отправить по адресу pygraphs.sber@gmail.com. Решения будут проверены до 19:00 2 марта. 

Также можно отправить решения до **19:00 27 февраля**. Тогда они будут проверены до 19:00 28 февраля и в случае наличия ошибок можно будет успеть их исправить до основного дедлайна.

В задании нужно реализовать 4 метода, описанных выше, каждый из них можно реализовывать независимо от остальных (хотя последний метод использует два предыдущих, для его реализации можно использовать лишь функции, реализация которых требуется в предыдущих методах). Первый метод оценивается в 13 баллов. Второй и третий метод оцениваются в 15 баллов каждый. Четвертый метод оценивается в 2 балла.

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

#!cp /content/drive/MyDrive/'Теория графов'/amazon0302.txt /content

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp /content/drive/MyDrive/'Теория графов'/amazon0302.txt /content

In [3]:
# В этом блоке мы загружаем граф из файла и приводим его в вид, удобный для работы

import networkx as nx

amazon = nx.read_edgelist("amazon0302.txt", create_using=nx.Graph(), nodetype=int, data=False)
amazon = nx.convert_node_labels_to_integers(amazon, ordering='decreasing degree')
nodes = amazon.number_of_nodes()

In [4]:
# В этом блоке собраны вспомогательные функции, которые нужны для реализации проекта
# Вам не обязательно их использовать, часть, где они необходимы, уже реализована в рамках проверок

# Эта функция получив на вход список возвращает список его индексов, упорядоченный по значению меток изначального списка
def index_sorted(a, reverse=False):
    return sorted(range(len(a)), key=lambda k: a[k], reverse=reverse)

# Эта функция позволяет выбрать ответ из посчитанных расстояний и сравнить его с целевым значением. 
# Она выбирает нужное количество вершин с минимальным расстоянием и находит число совпадений с удаленными ребрами.
# Здесь stat — это список с расстояниями, а samp — список выбираемых вершин с минимальным расстоянием.
def check_answer(stat, samp, reverse=False): 
    index_dist = index_sorted(stat, reverse)
    guess = index_dist[:len(samp)]
    return len(set(samp) & set(guess))

# Эта функция генерирует тестовый пример, удаляя данные ей ребра из графа.
# Здесь samp — количество удаляемых ребер.
# Здесь мы также конвертируем граф в словарь списков
def generate_dict(query, samp):
    graph = amazon.copy()
    for i in samp:
        graph.remove_edge(query, i)
    return nx.convert.to_dict_of_lists(graph)

In [5]:
# Здесь показан пример работы функции index_sorted

a = [3,-1,1,7,6]

print("Список индексов в порядке возрастания ячеек a:", index_sorted(a))
print("Список индексов в порядке убывания ячеек a:", index_sorted(a, True))

Список индексов в порядке возрастания ячеек a: [1, 2, 0, 4, 3]
Список индексов в порядке убывания ячеек a: [3, 4, 0, 2, 1]


In [6]:
# В отличие от функций в блоке выше, которые уже использованы в проверках,
# эта является подсказкой к тому, что вам самим нужно сделать после работы
# каждого из алгоритмов (в блоках SOLUTION). Конечно, эту функцию можно
# менять, если вам будет нужно.

def exclude_obvious_answers(graph, query, answers, worst_metric):
  '''
    Исключает `query` и её соседей из ответа.
    graph - коллекция списков связности (например, dict-of-lists)
    query - исключаемая вершина
    answers - коллекция метрик
    worst_metric - худший результат метрики (если меньше — лучше, то
      максимальная из возможных)
  '''
  answers[query] = worst_metric
  for neigh in graph[query]:
    answers[neigh] = worst_metric

In [7]:
# Здесь собраны полезные команды

a = [2,0,1,4,3]
b = [0, 5, 6, 3]

print("Преобразуем a в множество:",set(a))

print("Пересекаем два множества set(a) и set(b):",set(a) & set(b))

print("Возвращаем длину списка a:", len(a))

print("Возвращаем размер множества set(b):", len(set(b)))


Преобразуем a в множество: {0, 1, 2, 3, 4}
Пересекаем два множества set(a) и set(b): {0, 3}
Возвращаем длину списка a: 5
Возвращаем размер множества set(b): 4


In [8]:
# В этом блоке требуется реализовать метод числа общих соседей. 
# Функция в ячейке i списка common_neigh должна сохранить число общих соседей query и i. 
# Но есть одна тонкость: ячейку с номером query и с номерами ее соседей правильно обнулить, 
# а то нам будут рекомендовать соединить query с query или ее соседями
# Это можно сделать с помощью функции exclude_obvious_answers

def common_neighbours(graph, query):
    common_neigh = [0] * len(graph)
    ### BEGIN SOLUTION

    for (i,_) in enumerate(common_neigh):
        common_neigh[i] = len(list(set(graph[query]) & set(graph[i])))
        
    exclude_obvious_answers(graph, query, common_neigh, min(common_neigh))

    ### END SOLUTION
    return common_neigh

На примерах в следующем блоке вы можете протестировать ваши решения.
Важно: тесты нужны для самопроверки, оцениваться будет само решение.

In [9]:
query = 422
samp = [35561, 98891, 157171, 3060, 198304, 28054, 226896, 20673, 110999, 125875, 125877, 20342, 208996, 205186, 829, 189415, 212872, 164896, 104718, 78418]
graph = generate_dict(query, samp)

test_query = 377
test_samp = [202525, 196341, 169969, 29141, 159961, 38249, 101144, 1157, 40361, 99572, 64355, 127194, 109845, 217286, 125972, 77367, 6658, 26295, 47705, 200935]
test_graph = generate_dict(test_query, test_samp)

In [10]:
ans = common_neighbours(graph, query)

ind_sort = index_sorted(ans, reverse=True)[:len(samp)]
chck_ans = check_answer(ans, samp, reverse=True)
print("Ваши ответы:")
print("ТОП по числу соседей:", ind_sort)
print("Число совпадений с удалёнными рёбрами:", chck_ans)
print("Если получилось 5, возможно, вы забыли исключить query и соседей")
print("Если получилось 0, возможно, вы неправильно использовали exclude_obvious_answers")

print("Результаты тестов:")
assert chck_ans == 8
assert ind_sort == [829, 3060, 20673, 13141, 21150, 35561, 36377, 103988, 110999, 172699, 4863, 8961, 10572, 16003, 20342, 28054, 53201, 70084, 70323, 104718]
print("Сработало!")

Ваши ответы:
ТОП по числу соседей: [829, 3060, 20673, 13141, 21150, 35561, 36377, 103988, 110999, 172699, 4863, 8961, 10572, 16003, 20342, 28054, 53201, 70084, 70323, 104718]
Число совпадений с удалёнными рёбрами: 8
Если получилось 5, возможно, вы забыли исключить query и соседей
Если получилось 0, возможно, вы неправильно использовали exclude_obvious_answers
Результаты тестов:
Сработало!


In [12]:
ind_sort = index_sorted(common_neighbours(test_graph, test_query), reverse=True)[:len(test_samp)]
print("ТОП по числу соседей:", ind_sort)
assert ind_sort == [6658, 26295, 99789, 125972, 134665, 134666, 185446, 17364, 29519, 40361, 64355, 162514, 169969, 183713, 216721, 217286, 222821, 7693, 10838, 16638]
print("Сработало!")

ТОП по числу соседей: [6658, 26295, 99789, 125972, 134665, 134666, 185446, 17364, 29519, 40361, 64355, 162514, 169969, 183713, 216721, 217286, 222821, 7693, 10838, 16638]
Сработало!


In [13]:
# Здесь собраны полезные команды для реализации следующего блока

import random

print("Находим случайного соседа вершины 100:", random.choice(graph[100]))

Находим случайного соседа вершины 100: 175648


In [ ]:
# В этом блоке требуется реализовать метод случайных блужданий.
# Обратите внимание на массив used: его можно использовать для того, чтобы проверять, посещалась ли вершина в блуждании ранее
# Причем удобно не ставить там метку того, была ли посещена вершина в текущем блуждании.
# Вместо этого можно хранить номер последней итерации, на которой была посещена вершина, и сравниваем его с текущим.

import random

def hit_distance(graph, query, time=10):
    # инициализация статистик
    hit_dist = [0] * nodes  # искомые расстояния
    hit_times = [0] * nodes  # количество раз, когда вершина была достигнута в блуждании
    used = [0] * nodes  # последняя итерация, на которой вершина была достигнута в блуждании
    samples = nodes // time  # количество блужданий

    ### BEGIN SOLUTION

    ### END SOLUTION

    return hit_dist

In [ ]:
#####################################

In [21]:
time=10
query=422

hit_dist = [0] * nodes  # искомые расстояния
hit_times = [0] * nodes  # количество раз, когда вершина была достигнута в блуждании
used = [0] * nodes  # последняя итерация, на которой вершина была достигнута в блуждании
samples = nodes // time  # количество блужданий

node = query        # стартовая вершина
for i in range(samples):
    node = random.choice(graph[node])
    used[node] = i
    #print(f"{i}: node={node}")


In [19]:
samples

26211

In [ ]:
#####################################

Проверьте ваше решение.

In [ ]:
hd = hit_distance(graph, query)
hd_check = check_answer(hd, samp)
print("Совпадений с удалёнными рёбрами:", hd_check)
print("Если ответ 2 -- не исключили query и соседей, если 0 -- исключили неправильно")
assert hd_check >= 8
print("Сработало!")

In [ ]:
test_hd = hit_distance(test_graph, test_query)
hd_check = check_answer(test_hd, test_samp)
print("Совпадений с удалёнными рёбрами:", hd_check)
assert hd_check >= 9
print("Сработало!")

In [ ]:
# В этом блоке необходимо реализовать подсчет усеченных моментов достижения в вершину.
# Допишите функцию и постобработку (какие вершины точно не должны попасть в ответ?)
# В нашем тестовом графе нет петель, но если вы захотите потестировать свое решение на других графах,
# обратите внимание, что петля (ребро, идущее из вершины в саму себя) повышает степень вершины на 2

# P.S.: не рекомендуется делать функцию рекурсивной, потому что так вы будете
# вычислять заново одно и то же значение несколько раз. Подумайте, зачем вам дан
# массив tht и почему функция организована именно так.

def truncated_hitting_time(graph, query, time=10):
    tht = [[0 for _ in range(nodes)] for _ in range(time + 1)]
    for t in range(1, time + 1):
        for vert in range(nodes):
            if vert == query:
                continue
            if graph[vert]:
                ### BEGIN SOLUTION

                ### END SOLUTION
            tht[t][vert] += 1

    ### BEGIN SOLUTION

    ### END SOLUTION
    return tht[time]

Проверьте ваше решение.

In [ ]:
tht = truncated_hitting_time(graph, query)
ind_sort = index_sorted(tht)[:len(samp)]
chck_ans = check_answer(tht, samp)
print("Ваши ответы:")
print("ТОП по hitting-time:", ind_sort)
print("Совпадений с удалёнными рёбрами:", chck_ans)
print("Если 0, то, возможно, вы неправильно обработали query и соседей")
print("Результаты тестов:")
assert ind_sort == [164896, 254021, 110999, 212872, 20673, 172699, 3060, 104718, 205186, 194186, 35561, 36377, 829, 103988, 157171, 198304, 113283, 21150, 244935, 186662]
assert chck_ans == 11
print("Сработало!")

In [ ]:
test_tht = truncated_hitting_time(test_graph, test_query)
ind_sort = index_sorted(test_tht)[:len(test_samp)]
print("Ваши ответы:")
print("ТОП по hitting-time:", ind_sort)
print("Результаты тестов:")
assert ind_sort == [185446, 134665, 134666, 216721, 222821, 125972, 6658, 169969, 26295, 162514, 99789, 202525, 40361, 217286, 183713, 160748, 163128, 64355, 196341, 47705]
print("Сработало!")

In [ ]:
# В этом блоке требуется реализовать функцию, которая принимает две разные статистики и выдает новую,
# являющуюся суммой переданных

def sum_of_stats(first, second):
    ### BEGIN SOLUTION

    ### END SOLUTION

Проверьте ваше решение.

In [ ]:
assert check_answer(sum_of_stats(hd, tht), samp) >= 9

In [ ]:
assert check_answer(sum_of_stats(test_hd, test_tht), test_samp) >= 9